<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Breakout_Channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import logging
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress


logging.getLogger("tvDatafeed").setLevel(logging.ERROR)


# Zaman dilimi ismini döndürme fonksiyonu
def interval_name(interval):
    interval_map = {
        Interval.in_15_minute: '15 Dakika',
        Interval.in_30_minute: '30 Dakika',
        Interval.in_45_minute: '45 Dakika',
        Interval.in_1_hour: '1 Saat',
        Interval.in_2_hour: '2 Saat',
        Interval.in_4_hour: '4 Saat',
        Interval.in_daily: 'Günlük',
        Interval.in_weekly: 'Haftalık',
        Interval.in_monthly: 'Aylık'
    }
    return interval_map.get(interval, 'Günlük')

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimini seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}. {name}")

    choice = input("Seçiminiz (örnek: 6): ").strip()
    return intervals.get(choice, ('Günlük', Interval.in_daily))

# Trend Kanalı Hesaplama
def Trend_Channel(df):
    best_period = None
    best_r_value = 0
    periods = range(100, 201, 10)
    for period in periods:
        close_data = df['Close'].tail(period)
        x = np.arange(len(close_data))
        slope, intercept, r_value, _, _ = linregress(x, close_data)
        if abs(r_value) > abs(best_r_value):
            best_r_value = abs(r_value)
            best_period = period
    return best_period, best_r_value

# Trend Kırılmalarını Listeleme
def List_Trend_Breaks(Hisse, data, best_period, rval=0.85):
    close_data = data['Close'].tail(best_period)
    x_best_period = np.arange(len(close_data))
    slope_best_period, intercept_best_period, r_value_best_period, _, _ = linregress(x_best_period, close_data)
    trendline = slope_best_period * x_best_period + intercept_best_period
    upper_channel = trendline + (trendline.std() * 1.1)
    lower_channel = trendline - (trendline.std() * 1.1)

    upper_diff = upper_channel - close_data
    lower_diff = close_data - lower_channel
    last_upper_diff = upper_diff.iloc[-1]
    last_lower_diff = lower_diff.iloc[-1]

    if abs(r_value_best_period) > rval:
        if last_upper_diff < 0:
            return f'{Hisse}: Yukarı yönlü kırılım (R={abs(r_value_best_period):.2f}, Fark={last_upper_diff:.2f})', True, 'up'
        elif last_lower_diff < 0:
            return f'{Hisse}: Aşağı yönlü kırılım (R={abs(r_value_best_period):.2f}, Fark={last_lower_diff:.2f})', True, 'down'
    return None, False, None

# Hisse senetlerinin verilerini TradingView üzerinden çeker
def Stock_Prices_TradingView(symbol, interval, n_bars=1000):
    tv = TvDatafeed()
    data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=n_bars)
    data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
    return data

# Kullanıcıdan zaman dilimi seçme
selected_interval_name, selected_interval = get_interval_choice()

# Örnek hisse senetlerini (BIST) listeleme
Hisseler = get_all_symbols(market='turkey')  # BIST hisse senetlerini al
up_breaks = []
down_breaks = []

# Hisseleri kontrol etme
for hisse in Hisseler:
    try:
        # Veriyi TradingView'den çekme
        data = Stock_Prices_TradingView(hisse, selected_interval, n_bars=500)
        best_period, best_r_value = Trend_Channel(data)
        result, status, direction = List_Trend_Breaks(hisse, data, best_period)

        if result:
            if direction == 'up':
                up_breaks.append(result)
            elif direction == 'down':
                down_breaks.append(result)
        print(f'{hisse} kontrol ediliyor: {status}')
    except Exception as e:
        print(f'Hisse {hisse} için hata: {e}')
        continue

# Sonuçları yazdırma
if up_breaks:
    print("\nYukarı yönlü kırılımlar:")
    for break_info in up_breaks:
        print(break_info)
else:
    print("\nYukarı yönlü kırılım tespit edilmedi.")

if down_breaks:
    print("\nAşağı yönlü kırılımlar:")
    for break_info in down_breaks:
        print(break_info)
else:
    print("\nAşağı yönlü kırılım tespit edilmedi.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-7537w03o
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-7537w03o
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=a27b37bb0595c8858ec013e32e5b29819bde5c8072d64629a633f586da1eb8f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-fo7zitv0/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
Lütfen zaman dilimini seçin:
1. 15 Dakika
2. 30 Dakika
3. 45 Dakika
4. 1 Saat
5. 2 Saat
6. 4 Saat
7. Günlük
8. Haftalık
9. Aylık
Seçiminiz (örnek: 6): 7
BIST:UFUK kontrol ediliyor: False
BIST:ALGYO kontrol ediliyor: False
BIST:QTEMZ kontrol ediliyor: False
BIST:ES

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


BIST:QNBTR kontrol ediliyor: False
Hisse BIST:SKYLP için hata: 'NoneType' object has no attribute 'rename'
BIST:DMRGD kontrol ediliyor: False
BIST:MIATK kontrol ediliyor: False
BIST:YIGIT kontrol ediliyor: False
BIST:TGSAS kontrol ediliyor: False
BIST:DURDO kontrol ediliyor: False
BIST:YKBNK kontrol ediliyor: False
BIST:ERBOS kontrol ediliyor: False
BIST:KGYO kontrol ediliyor: True

Yukarı yönlü kırılımlar:
BIST:CIMSA: Yukarı yönlü kırılım (R=0.88, Fark=-0.93)
BIST:PRKAB: Yukarı yönlü kırılım (R=0.86, Fark=-0.06)
BIST:PASEU: Yukarı yönlü kırılım (R=0.96, Fark=-0.26)
BIST:AKCNS: Yukarı yönlü kırılım (R=0.87, Fark=-12.80)
BIST:KFEIN: Yukarı yönlü kırılım (R=0.85, Fark=-10.42)

Aşağı yönlü kırılımlar:
BIST:ENERY: Aşağı yönlü kırılım (R=0.91, Fark=-1.35)
BIST:BMSCH: Aşağı yönlü kırılım (R=0.85, Fark=-0.76)
BIST:KGYO: Aşağı yönlü kırılım (R=0.87, Fark=-2.72)
